In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Packages

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer,WordNetLemmatizer
from string import punctuation
from wordcloud import WordCloud
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

# Preparing Data

In [ ]:
#INPUT DATA
train = pd.read_csv("../input/movie-review-sentiment-analysis-kernels-only/train.tsv.zip",sep="\t")
test = pd.read_csv("../input/movie-review-sentiment-analysis-kernels-only/test.tsv.zip",sep="\t")
sub = pd.read_csv("../input/movie-review-sentiment-analysis-kernels-only/sampleSubmission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

#  Exploratory Data

**Jumlah Ulasan**

In [ ]:
#Count Of Tweets
sns.countplot(train['Sentiment'],palette='rocket_r')
plt.title("No of Tweet Sentiments")

Label sentiment

0 - negative

1 - somewhat negative

2 - neutral

3 - somewhat positive

4 - positive

> **Rara-rata Kata/Frasa per Kalimat**

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))
plt.subplot(1,2,1)
sns.distplot(train.groupby('SentenceId')['PhraseId'].count())
plt.title("Avg Phrases Per Sentence in Train")
plt.subplot(1,2,2)
sns.distplot(test.groupby('SentenceId')['PhraseId'].count())
plt.title("Avg Phrases Per Sentence in Test")

print("Avg Phrases Per Sentence in Train: ",round(train.groupby('SentenceId')['PhraseId'].count().mean()))
print("Avg Phrases Per Sentence in Test: ",round(test.groupby('SentenceId')['PhraseId'].count().mean()))

**Jumlah Kata dalam Kalimat**

In [ ]:
fig,ax = plt.subplots(figsize=(22,5))
fig.suptitle("Avg Words In Phrases",fontsize=16)

plt.subplot(1,5,1)
sns.distplot(train[train['Sentiment']==0]['Phrase'].str.split().apply(lambda x:len(x)))
plt.title("Sentiment 0")
print("Avg Words in Phrases with Sentiment 0: ",round(train[train['Sentiment']==0]['Phrase'].str.split().apply(lambda x:len(x)).mean()))

plt.subplot(1,5,2)
sns.distplot(train[train['Sentiment']==1]['Phrase'].str.split().apply(lambda x:len(x)))
plt.title("Sentiment 1")
print("Avg Words in Phrases with Sentiment 1: ",round(train[train['Sentiment']==1]['Phrase'].str.split().apply(lambda x:len(x)).mean()))

plt.subplot(1,5,3)
sns.distplot(train[train['Sentiment']==2]['Phrase'].str.split().apply(lambda x:len(x)))
plt.title("Sentiment 2")
print("Avg Words in Phrases with Sentiment 2: ",round(train[train['Sentiment']==2]['Phrase'].str.split().apply(lambda x:len(x)).mean()))

plt.subplot(1,5,4)
sns.distplot(train[train['Sentiment']==3]['Phrase'].str.split().apply(lambda x:len(x)))
plt.title("Sentiment 3")
print("Avg Words in Phrases with Sentiment 3: ",round(train[train['Sentiment']==3]['Phrase'].str.split().apply(lambda x:len(x)).mean()))

plt.subplot(1,5,5)
sns.distplot(train[train['Sentiment']==4]['Phrase'].str.split().apply(lambda x:len(x)))
plt.title("Sentiment 4")
print("Avg Words in Phrases with Sentiment 4: ",round(train[train['Sentiment']==4]['Phrase'].str.split().apply(lambda x:len(x)).mean()))

**Rataan Char (Huruf) dalam Kata/Frasa**

In [ ]:
#Avg Characters in Phrases
fig,ax = plt.subplots(figsize=(22,5))
fig.suptitle("Avg Characters In Phrases",fontsize=16)

plt.subplot(1,5,1)
sns.distplot(train[train['Sentiment']==0]['Phrase'].str.len())
plt.title("Sentiment 0")
print("Avg Characters in Phrases with Sentiment 0: ",round(train[train['Sentiment']==0]['Phrase'].str.len().mean()))

plt.subplot(1,5,2)
sns.distplot(train[train['Sentiment']==1]['Phrase'].str.len())
plt.title("Sentiment 1")
print("Avg Characters in Phrases with Sentiment 1: ",round(train[train['Sentiment']==1]['Phrase'].str.len().mean()))

plt.subplot(1,5,3)
sns.distplot(train[train['Sentiment']==2]['Phrase'].str.len())
plt.title("Sentiment 2")
print("Avg Characters in Phrases with Sentiment 2: ",round(train[train['Sentiment']==2]['Phrase'].str.len().mean()))

plt.subplot(1,5,4)
sns.distplot(train[train['Sentiment']==3]['Phrase'].str.len())
plt.title("Sentiment 3")
print("Avg Characters in Phrases with Sentiment 3: ",round(train[train['Sentiment']==3]['Phrase'].str.len().mean()))

plt.subplot(1,5,5)
sns.distplot(train[train['Sentiment']==4]['Phrase'].str.len())
plt.title("Sentiment 4")
print("Avg Characters in Phrases with Sentiment 4: ",round(train[train['Sentiment']==4]['Phrase'].str.len().mean()))

In [ ]:
#Checking datatypes,no. of elements and empty data columns
train.dtypes
train.shape

In [ ]:
train.info() #No null entry in training dataset

In [ ]:
test.info()

# Data Analysis and Visualization

In [ ]:
train.columns

In [ ]:
# Sentiments vs Pharses Count
senti_phrase=train.groupby(['Sentiment'])['PhraseId'].count().sort_values(ascending=False)
print("Phrases count wr.t various Sentiments:\n ",senti_phrase)

In [ ]:
# Sentiment label-vs-Sentences
senti_sentence=train.groupby(['Sentiment'])['SentenceId'].nunique().sort_values(ascending=False)
senti_sentence

This means the movies with Neutral Sentiment types(i.e Sentiment label-2) had highest number pf phrase sentences and Sentiment label-0 i.e Negative sentiments had minimum number of sentences used.

In [ ]:
# Plotting relationship between Sentiment labels vs Sentences and Pharses used
sns.set_style("darkgrid")
sns.set_context("talk")
plt.figure(figsize=(15,10))

x=senti_phrase.index
y=senti_phrase.values

plt.subplot(2,1,1)
plt.bar(x,y,color='c')
plt.title('Sentiment vs Phrases Used')

x1=senti_sentence.index
y1=senti_sentence.values

plt.subplot(2,1,2)
plt.bar(x1,y1,color='y')
plt.title('Sentiment vs Sentences')

**Top words used in whole phrases records**
Most frequently used Words in Phrases

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
corpus = ' '.join(train['Phrase'])
corpus = corpus.replace('.', '. ')
wordcloud= WordCloud(stopwords=STOPWORDS,background_color='white', width=2400,height=2000,).generate(corpus)
plt.figure(figsize=(15,20))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
#WordCloud
fig,ax = plt.subplots(figsize=(20,40))
plt.axis('off')

plt.subplot(5,1,1)
text = " ".join(train[train['Sentiment']==0]['Phrase'])
text = text.replace('.', '. ')
wordcloud = WordCloud(stopwords=STOPWORDS, max_font_size = None,background_color='white',width=1000,height=1000).generate(text)
plt.title("WordCloud for Sentiment 0")
plt.axis('off')
plt.imshow(wordcloud)

plt.subplot(5,1,2)
text = " ".join(train[train['Sentiment']==1]['Phrase'])
text = text.replace('.', '. ')
wordcloud = WordCloud(stopwords=STOPWORDS, max_font_size = None,background_color='white',width=1000,height=1000).generate(text)
plt.title("WordCloud for Sentiment 1")
plt.axis('off')
plt.imshow(wordcloud)

plt.subplot(5,1,3)
text = " ".join(train[train['Sentiment']==2]['Phrase'])
text = text.replace('.', '. ')
wordcloud = WordCloud(stopwords=STOPWORDS, max_font_size = None,background_color='white',width=1000,height=1000).generate(text)
plt.title("WordCloud for Sentiment 2")
plt.axis('off')
plt.imshow(wordcloud)

plt.subplot(5,1,4)
text = " ".join(train[train['Sentiment']==3]['Phrase'])
text = text.replace('.', '. ')
wordcloud = WordCloud(stopwords=STOPWORDS, max_font_size = None,background_color='white',width=1000,height=1000).generate(text)
plt.title("WordCloud for Sentiment 3")
plt.axis('off')
plt.imshow(wordcloud)

plt.subplot(5,1,5)
text = " ".join(train[train['Sentiment']==4]['Phrase'])
text = text.replace('.', '. ')
wordcloud = WordCloud(stopwords=STOPWORDS, max_font_size = None,background_color='white',width=1000,height=1000).generate(text)
plt.title("WordCloud for Sentiment 4")
plt.axis('off')
plt.imshow(wordcloud)

# TF-IDF Implementation

**TF-IDF and uncleaned dataset**

In [ ]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
Stopwords = list(ENGLISH_STOP_WORDS)
Stopwords.extend(['movie','movies','film','nt','rrb','lrb',
                      'make','work','like','story','time','little',])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

vectorizer = TfidfVectorizer(ngram_range=(1, 3), tokenizer=tokenizer.tokenize,  stop_words=Stopwords)
full_text = list(train['Phrase'].values) + list(test['Phrase'].values)
#full_text

vectorizer.fit(full_text)
train_vectorized = vectorizer.transform(train['Phrase'])
test_vectorized = vectorizer.transform(test['Phrase'])

**Dimensi data train setelah dilakukan TF-IDF Vectorizer**

In [ ]:
np.shape(train_vectorized)

In [ ]:
X = train_vectorized
y = train.Sentiment.values


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import time

# create a stratified split of the data and a 80/20 split.
xtrain, xvalid, ytrain, yvalid = train_test_split(X, y, stratify=y, test_size=0.2, shuffle=True)

In [ ]:
ml_default_performance_metrics_df = pd.DataFrame(columns=['accuracy','F1-score','training-time'], index=['LinearSVC', 'BernoulliNB', 'MultinomialNB'])

# Multiclass Sentiment Analysis

**LINEAR SVC**

In [ ]:
from sklearn.svm import LinearSVC

start_time = time.time()
    
print()
print("Evaluation of LinearSVC, multi_class='ovr', with train-test split:")

clf_LinearSVC = LinearSVC(multi_class='ovr')
clf_LinearSVC.fit(xtrain, ytrain)
predictions = clf_LinearSVC.predict(xvalid)
print(classification_report(yvalid, predictions))

print()
print("accuracy_score", accuracy_score(yvalid, predictions))

print()
print("Weighted Averaged validation metrics")
print("precision_score", precision_score(yvalid, predictions, average='weighted'))
print("recall_score", recall_score(yvalid, predictions, average='weighted'))
print("f1_score", f1_score(yvalid, predictions, average='weighted'))


ml_default_performance_metrics_df.loc['LinearSVC']['training-time'] = time.time() - start_time
ml_default_performance_metrics_df.loc['LinearSVC']['accuracy'] = accuracy_score(yvalid, predictions)
ml_default_performance_metrics_df.loc['LinearSVC']['F1-score'] = f1_score(yvalid, predictions, average='weighted')

print()
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_confusion_matrix(yvalid, predictions)

print()
print("elapsed time in seconds: ", time.time() - start_time)
print()

**BernoulliNB**

In [ ]:
from sklearn.naive_bayes import BernoulliNB

start_time = time.time()
    
print()
print("Evaluation of BernoulliNB with train-test split:")

clf_BernoulliNB = BernoulliNB()
clf_BernoulliNB.fit(xtrain, ytrain)
predictions = clf_BernoulliNB.predict(xvalid)
print(classification_report(yvalid, predictions))

print()
print("accuracy_score", accuracy_score(yvalid, predictions))

print()
print("Weighted Averaged validation metrics")
print("precision_score", precision_score(yvalid, predictions, average='weighted'))
print("recall_score", recall_score(yvalid, predictions, average='weighted'))
print("f1_score", f1_score(yvalid, predictions, average='weighted'))


ml_default_performance_metrics_df.loc['BernoulliNB']['training-time'] = time.time() - start_time
ml_default_performance_metrics_df.loc['BernoulliNB']['accuracy'] = accuracy_score(yvalid, predictions)
ml_default_performance_metrics_df.loc['BernoulliNB']['F1-score'] = f1_score(yvalid, predictions, average='weighted')

print()
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_confusion_matrix(yvalid, predictions)

print()
print("elapsed time in seconds: ", time.time() - start_time)
print()

**MultinomialNB**

In [ ]:
from sklearn.naive_bayes import MultinomialNB

start_time = time.time()
    
print()
print("Evaluation of MultinomialNB with train-test split:")

clf_MultinomialNB = MultinomialNB()
clf_MultinomialNB.fit(xtrain, ytrain)
predictions = clf_MultinomialNB.predict(xvalid)
print(classification_report(yvalid, predictions))

print()
print("accuracy_score", accuracy_score(yvalid, predictions))

print()
print("Weighted Averaged validation metrics")
print("precision_score", precision_score(yvalid, predictions, average='weighted'))
print("recall_score", recall_score(yvalid, predictions, average='weighted'))
print("f1_score", f1_score(yvalid, predictions, average='weighted'))


ml_default_performance_metrics_df.loc['MultinomialNB']['training-time'] = time.time() - start_time
ml_default_performance_metrics_df.loc['MultinomialNB']['accuracy'] = accuracy_score(yvalid, predictions)
ml_default_performance_metrics_df.loc['MultinomialNB']['F1-score'] = f1_score(yvalid, predictions, average='weighted')

print()
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_confusion_matrix(yvalid, predictions)

print()
print("elapsed time in seconds: ", time.time() - start_time)
print()

**Performa berdasarkan Akurasi**

In [ ]:
ml_default_performance_metrics_df.sort_values(by="accuracy", ascending=False)

In [ ]:
sns.set(rc={'figure.figsize':(15.27,6.27)})
ml_default_performance_metrics_df.sort_values(by="accuracy", ascending=False).accuracy.plot(kind="bar")

**Performa berdasarkan F1-Score**

In [ ]:
ml_default_performance_metrics_df.sort_values(by="F1-score", ascending=True)

In [ ]:
sns.set(rc={'figure.figsize':(15.27,6.27)})
ml_default_performance_metrics_df.sort_values(by="F1-score", ascending=False)["F1-score"].plot(kind="bar")

**Performa berdasarkan Waktu Latih**

In [ ]:
ml_default_performance_metrics_df.sort_values(by="training-time", ascending=True)

In [ ]:
sns.set(rc={'figure.figsize':(15.27,6.27)})
ml_default_performance_metrics_df.sort_values(by="training-time", ascending=True)["training-time"].plot(kind="bar")

# Submission

In [ ]:
features = vectorizer.fit_transform(train.Phrase).toarray()
labels = train.Sentiment
features.shape

In [ ]:
features_test = vectorizer.transform(test.Phrase)
features_test.shape

In [ ]:
model = clf_LinearSVC
model.fit(features, labels)
pred = model.predict(features_test)
submission = test
submission['sentiment']=pred
submission.drop(['SentenceId', 'Phrase'], axis=1, inplace=True)


In [ ]:
submission